In [4]:
from numba.cuda.printimpl import print_item

from helper.steam_scraper import *
from helper.utils import *

# General modules
import os


# Paths
root_dir = r'C:\Users\fbohm\Desktop\Projects\DataScience\cluster_analysis'
path_input = os.path.join(root_dir, "Data", "Into the Dead Our Darkest Days_1511resposnes.xlsx")

path_steam_reviews = os.path.join(root_dir, "Data", "steam_reviews.json")


In [7]:
params = {
    'json': 1,
    'filter': 'recent',
    'language': 'all',
    'day_range': 9223372036854775807,
    'review_type': 'all',
    'purchase_type': 'all'
}

appid = 1128810
reviews = get_n_reviews(str(appid), 1000, params)

save_to_json(reviews, path_steam_reviews)

2024-11-26 12:10:27,115 - INFO - Retrieved 100 reviews so far.
2024-11-26 12:10:27,592 - INFO - Retrieved 200 reviews so far.
2024-11-26 12:10:28,038 - INFO - Retrieved 300 reviews so far.
2024-11-26 12:10:28,484 - INFO - Retrieved 400 reviews so far.
2024-11-26 12:10:28,929 - INFO - Retrieved 500 reviews so far.
2024-11-26 12:10:29,395 - INFO - Retrieved 600 reviews so far.
2024-11-26 12:10:29,914 - INFO - Retrieved 700 reviews so far.
2024-11-26 12:10:30,332 - INFO - Retrieved 800 reviews so far.
2024-11-26 12:10:30,807 - INFO - Retrieved 900 reviews so far.
2024-11-26 12:10:31,273 - INFO - Retrieved 1000 reviews so far.
2024-11-26 12:10:31,318 - INFO - Data successfully saved to C:\Users\fbohm\Desktop\Projects\DataScience\cluster_analysis\Data\steam_reviews.json


In [6]:
len(reviews)

1000

# Fetch reviews for a specific time interval

In [15]:
import requests
import logging
from datetime import datetime

# Configure logging
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")
logger = logging.getLogger(__name__)

from helper.utils import save_to_json


###########################################

def get_user_reviews(review_appid, params):
    user_review_url = f'https://store.steampowered.com/appreviews/{review_appid}'
    req_user_review = requests.get(
        user_review_url,
        params=params
    )

    if req_user_review.status_code != 200:
        print(f'Fail to get response. Status code: {req_user_review.status_code}')
        return {"success": 2}

    try:
        user_reviews = req_user_review.json()
    except:
        return {"success": 2}

    return user_reviews


def fetch_reviews_time_stamp(appid, params, start_time=None, end_time=None):
    """Fetches reviews with optional time filtering."""
    passed_start_time = not start_time  # If no start_time, assume it's passed
    passed_end_time = not end_time  # If no end_time, assume it's passed
    selected_reviews = []

    while not passed_start_time or not passed_end_time:
        reviews_response = get_user_reviews(appid, params)

        # Not successful response
        if reviews_response["success"] != 1:
            logger.error("Steam API returned not 1 as success. Unable to fetch reviews.")
            break

        if reviews_response["query_summary"]['num_reviews'] == 0:
            logger.info("No more reviews available.")
            break

        for review in reviews_response["reviews"]:
            timestamp_created = review['timestamp_created']

            # Time filtering logic
            if end_time and timestamp_created > end_time.timestamp():
                logger.info("Review out of date range")
                continue
            if start_time and timestamp_created < start_time.timestamp():
                logger.info("Review within RANGE")
                passed_start_time = True
                break

            selected_reviews.append({
                'recommendationid': review['recommendationid'],
                'author_steamid': review['author']['steamid'],
                'playtime_at_review_minutes': review['author']['playtime_at_review'],
                'playtime_forever_minutes': review['author']['playtime_forever'],
                'playtime_last_two_weeks_minutes': review['author']['playtime_last_two_weeks'],
                'last_played': review['author']['last_played'],
                'review_text': review['review'],
                'timestamp_created': timestamp_created,
                'timestamp_updated': review['timestamp_updated'],
                'voted_up': review['voted_up'],
                'votes_up': review['votes_up'],
                'votes_funny': review['votes_funny'],
                'weighted_vote_score': review['weighted_vote_score'],
                'steam_purchase': review['steam_purchase'],
                'received_for_free': review['received_for_free'],
                'written_during_early_access': review['written_during_early_access'],
            })

        # Update cursor to fetch next page
        cursor = reviews_response.get('cursor', '')
        if not cursor:
            logger.info(f"No more pages available. (last cursor reached {cursor})")
            break
        params['cursor'] = cursor
        print("Cursor: ", cursor)
        print("rev:" , reviews[0])

    return selected_reviews



In [19]:
import test

appid = 455690

params = {
        'json' : 1,
        'filter' : 'all',
        'language' : 'all',
  #      'day_range': 9223372036854775807,
        'start_date': 1514721661,
        'end_date': 1546257661,
        'review_type' : 'recent',
        'purchase_type' : 'all'
        }

# time interval
end_time = datetime(2024, 11, 1, 0, 0, 0)
start_time = datetime(2018, 1, 1, 0, 0, 0)

reviews = fetch_reviews_time_stamp(appid, params, start_time, end_time)
save_to_json(reviews, 'Pixel_Puzzles_Junior_Jigsaw_reviews.json')
print(len(reviews))

2024-11-26 15:12:23,211 - INFO - Review out of date range
2024-11-26 15:12:23,211 - INFO - Review out of date range


Cursor:  AoMFQFaAAAAAAAAFP+AAAAAAAAB0yKiqBQ==
rev: {'recommendationid': '142876904', 'author_steamid': '76561197973230221', 'playtime_at_review_minutes': 194, 'playtime_forever_minutes': 194, 'playtime_last_two_weeks_minutes': 0, 'last_played': 1690409905, 'review_text': '+++ easy, fast & cheap (Sale) 100 % Steam Achievements \n\n\nhttps://steamcommunity.com/sharedfiles/filedetails/?id=3010367091\n\n++ good for Kids to Puzzle (9,20,30,42,56 Pieces x 50 Puzzles)\n\nhttps://steamcommunity.com/sharedfiles/filedetails/?id=3010367427\n\n+ 12 Trading Cards (5 would be better, Point Store Items cloud be cuter)\n\nGood for Achievement Hunter, Puzzle Fan Kids & Games Collectors\n\nhttps://store.steampowered.com/app/455690/Pixel_Puzzles_Junior_Jigsaw/', 'timestamp_created': 1690437365, 'timestamp_updated': 1721400143, 'voted_up': True, 'votes_up': 75, 'votes_funny': 2, 'weighted_vote_score': '0.887793660163879395', 'steam_purchase': True, 'received_for_free': False, 'written_during_early_access'

2024-11-26 15:12:23,594 - INFO - Review out of date range
2024-11-26 15:12:23,594 - INFO - Review out of date range


Cursor:  AoMFQH2B0OAAAAAFP+FrHmAAAABxztikBQ==
rev: {'recommendationid': '142876904', 'author_steamid': '76561197973230221', 'playtime_at_review_minutes': 194, 'playtime_forever_minutes': 194, 'playtime_last_two_weeks_minutes': 0, 'last_played': 1690409905, 'review_text': '+++ easy, fast & cheap (Sale) 100 % Steam Achievements \n\n\nhttps://steamcommunity.com/sharedfiles/filedetails/?id=3010367091\n\n++ good for Kids to Puzzle (9,20,30,42,56 Pieces x 50 Puzzles)\n\nhttps://steamcommunity.com/sharedfiles/filedetails/?id=3010367427\n\n+ 12 Trading Cards (5 would be better, Point Store Items cloud be cuter)\n\nGood for Achievement Hunter, Puzzle Fan Kids & Games Collectors\n\nhttps://store.steampowered.com/app/455690/Pixel_Puzzles_Junior_Jigsaw/', 'timestamp_created': 1690437365, 'timestamp_updated': 1721400143, 'voted_up': True, 'votes_up': 75, 'votes_funny': 2, 'weighted_vote_score': '0.887793660163879395', 'steam_purchase': True, 'received_for_free': False, 'written_during_early_access'

2024-11-26 15:12:24,179 - INFO - Review out of date range
2024-11-26 15:12:24,194 - INFO - Review out of date range


...

2024-11-26 15:12:39,732 - INFO - Review out of date range
2024-11-26 15:12:39,748 - INFO - Review out of date range


Cursor:  AoMFQH2B0OAAAAAFP+FrHmAAAABxztikBQ==
rev: {'recommendationid': '142876904', 'author_steamid': '76561197973230221', 'playtime_at_review_minutes': 194, 'playtime_forever_minutes': 194, 'playtime_last_two_weeks_minutes': 0, 'last_played': 1690409905, 'review_text': '+++ easy, fast & cheap (Sale) 100 % Steam Achievements \n\n\nhttps://steamcommunity.com/sharedfiles/filedetails/?id=3010367091\n\n++ good for Kids to Puzzle (9,20,30,42,56 Pieces x 50 Puzzles)\n\nhttps://steamcommunity.com/sharedfiles/filedetails/?id=3010367427\n\n+ 12 Trading Cards (5 would be better, Point Store Items cloud be cuter)\n\nGood for Achievement Hunter, Puzzle Fan Kids & Games Collectors\n\nhttps://store.steampowered.com/app/455690/Pixel_Puzzles_Junior_Jigsaw/', 'timestamp_created': 1690437365, 'timestamp_updated': 1721400143, 'voted_up': True, 'votes_up': 75, 'votes_funny': 2, 'weighted_vote_score': '0.887793660163879395', 'steam_purchase': True, 'received_for_free': False, 'written_during_early_access'

2024-11-26 15:12:40,165 - INFO - Review out of date range
2024-11-26 15:12:40,165 - INFO - Review out of date range


Cursor:  AoMFQH2B0OAAAAAFP+FrHmAAAABxztikBQ==
rev: {'recommendationid': '142876904', 'author_steamid': '76561197973230221', 'playtime_at_review_minutes': 194, 'playtime_forever_minutes': 194, 'playtime_last_two_weeks_minutes': 0, 'last_played': 1690409905, 'review_text': '+++ easy, fast & cheap (Sale) 100 % Steam Achievements \n\n\nhttps://steamcommunity.com/sharedfiles/filedetails/?id=3010367091\n\n++ good for Kids to Puzzle (9,20,30,42,56 Pieces x 50 Puzzles)\n\nhttps://steamcommunity.com/sharedfiles/filedetails/?id=3010367427\n\n+ 12 Trading Cards (5 would be better, Point Store Items cloud be cuter)\n\nGood for Achievement Hunter, Puzzle Fan Kids & Games Collectors\n\nhttps://store.steampowered.com/app/455690/Pixel_Puzzles_Junior_Jigsaw/', 'timestamp_created': 1690437365, 'timestamp_updated': 1721400143, 'voted_up': True, 'votes_up': 75, 'votes_funny': 2, 'weighted_vote_score': '0.887793660163879395', 'steam_purchase': True, 'received_for_free': False, 'written_during_early_access'

KeyboardInterrupt: 

In [12]:
len(reviews)

1000

In [13]:
reviews

[{'recommendationid': '179762031',
  'author': {'steamid': '76561198803712938',
   'num_games_owned': 144,
   'num_reviews': 13,
   'playtime_forever': 61,
   'playtime_last_two_weeks': 0,
   'playtime_at_review': 61,
   'last_played': 1584657345},
  'language': 'spanish',
  'review': 'morks',
  'timestamp_created': 1732496038,
  'timestamp_updated': 1732496038,
  'voted_up': False,
  'votes_up': 0,
  'votes_funny': 0,
  'weighted_vote_score': 0,
  'comment_count': 0,
  'steam_purchase': False,
  'received_for_free': False,
  'written_during_early_access': False,
  'primarily_steam_deck': False},
 {'recommendationid': '179761773',
  'author': {'steamid': '76561198026378271',
   'num_games_owned': 106,
   'num_reviews': 2,
   'playtime_forever': 15391,
   'playtime_last_two_weeks': 1113,
   'playtime_at_review': 15304,
   'last_played': 1732536796},
  'language': 'danish',
  'review': 'meh',
  'timestamp_created': 1732495632,
  'timestamp_updated': 1732495632,
  'voted_up': True,
  'vot